## Final SpIES High-z Quasar Selection

Notebook performing selection of $3.5<z<5$ quasars from SDSS+SpIES data.

First load the training data, then instantiate and train the algorithm; see [https://github.com/gtrichards/QuasarSelection/blob/master/SpIESHighzCandidateSelection.ipynb](https://github.com/gtrichards/QuasarSelection/blob/master/SpIESHighzCandidateSelection.ipynb)

In [1]:
%matplotlib inline
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
data = Table.read('GTR-ADM-QSO-ir-testhighz_findbw_lup_2016_starclean.fits')

# X is in the format need for all of the sklearn tools, it just has the colors
X = np.vstack([ data['ug'], data['gr'], data['ri'], data['iz'], data['zs1'], data['s1s2']]).T
y = np.array(data['labels'])

/Users/gtr/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, max_depth=15, min_samples_split=2, n_jobs=-1, random_state=42)
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

Second, load the test data

### Test Data
Test set data set was made as follows (see 18 April 2016 README entry):
   
    maketest_2016.py

    Output is:
    classifiers_out = open('GTR-ADM-QSO-ir_classifiers_good_test_2016.dat','w') 
    others_out= open('GTR-ADM-QSO-ir_others_good_test_2016.dat','w')
    czr_out = open('GTR-ADM-QSO-ir_photoz_in7_good_test_2016.dat','w')

Really need the first two files combined (so that we have both RA/Dec and colors in one place).

But couldn't merge them with TOPCAT or STILTS.  So had to break them into 3 pieces (with TOPCAT),
then used `combine_test_files_STILTS.py` to merge them together (just changing the input/output file names by hand).  

Actually ran this on dirac so that I'd have more memory than on quasar.  Copied the output files back to quasar and merged them together with TOPCAT.

So<br>
`GTR-ADM-QSO-ir_others_good_test_2016a.dat` + `GTR-ADM-QSO-ir_classifiers_good_test_2016a.dat`<br>
gives<br>
`GTR-ADM-QSO-ir_good_test_2016a.dat`<br>
(and so on for "b" and "c").

Then<br>
`GTR-ADM-QSO-ir_good_test_2016a.dat` + `GTR-ADM-QSO-ir_good_test_2016b.dat` + `GTR-ADM-QSO-ir_good_test_2016c.dat`<br>
gives<br>
`GTR-ADM-QSO-ir_good_test_2016.dat`

N.B.  Had to delete the intermediate files from quasar to save disk space.

Now read in the test file and convert it to an appropriate array format for sklearn.

In [8]:
data2 = Table.read('GTR-ADM-QSO-ir_good_test_2016.fits')
#data2 = Table.read('GTR-ADM-QSO-ir_classifiers_good_test_2016.fits')
#data2 = Table.read('gtr-adm-qso-ir_photoz_in7_good_test_2016.fits')

Xtest = np.vstack([ data2['ug'], data2['gr'], data2['ri'], data2['iz'], data2['zs1'], data2['s1s2']]).T

print len(Xtest),Xtest[:5]

50939704 [[  3.81193849e-02   3.65796465e-01   5.31836570e-02  -4.89434865e-02
    6.28884801e-01   6.24938477e-01]
 [  5.08303480e-01   1.11420093e-01   8.65531714e-02   1.03706865e-01
    4.21802021e-01   3.36181068e-01]
 [  1.65061871e+00   1.13939853e-01   1.54225782e-01   4.41376234e-01
    4.62311618e-01   4.68052444e-01]
 [  3.68773847e-01   2.59072437e-01   1.81469411e-01   1.44563753e-02
    7.14639858e-01   7.00231807e-01]
 [ -1.32032589e-02   1.96339791e-01   7.44004894e-02   5.95867945e-04
    1.46086353e+00   3.08763164e-01]]


### Quasar Candidates
Finally, do the classification and output the test file, including the predicted labels.

In [9]:
ypred = clf.predict(Xtest)

In [10]:
print ypred[:5]
print len(ypred)

[1 1 1 1 1]
50939704


In [12]:
data2['ypred'] = ypred
data2.write('GTR-ADM-QSO-ir_good_test_2016_out.fits', format='fits')

This gives about 27,000 candidates in all and about 3000 on Stripe 82.

You can get the output file from
[http://www.physics.drexel.edu/~gtr/outgoing/spies/GTR-ADM-QSO-ir_good_test_2016_out.fits.bz2](http://www.physics.drexel.edu/~gtr/outgoing/spies/GTR-ADM-QSO-ir_good_test_2016_out.fits.bz2)


In [14]:
mask = (data2['ypred']==0)
quasars = data2[mask]
print len(quasars)

27225
